In [54]:
import os
import struct
import numpy as np
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte'% kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte'% kind) 
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)
    
    return images, labels

X_train, y_train = load_mnist('mnist', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))
X_test, y_test = load_mnist('mnist', kind='t10k')

Rows: 60000, columns: 784


In [65]:
import theano
from keras.datasets import mnist

theano.config.floatX = 'float32'

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

X_train = X_train.astype(theano.config.floatX)
X_test = X_test.astype(theano.config.floatX)

In [66]:
from keras.utils import np_utils

y_train_ohe = np_utils.to_categorical(y_train)

print y_train_ohe[:3]

[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [68]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.layers import Dropout

np.random.seed(1)

model = Sequential()

model.add(Dense(input_dim = X_train.shape[1],
                output_dim = 300,
                init = 'uniform',
                activation = 'tanh'))

model.add(Dropout(0.25))

model.add(Dense(input_dim=300,
                output_dim=100,
                init='uniform',
                activation='tanh'))

model.add(Dropout(0.2))

model.add(Dense(input_dim = 100,
                output_dim = y_train_ohe.shape[1],
                init = 'uniform',
                activation = 'softmax'))

sgd = SGD(lr = 0.001, decay = 1e-7, momentum = 0.9)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd)

In [69]:
history = model.fit(X_train, 
          y_train_ohe,
          nb_epoch = 50,
          batch_size = 100,
          verbose = 1,
          validation_split = 0.1,
          show_accuracy = True)
# print(history.history.keys())

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 4s - loss: 1.2493 - val_loss: 0.5262
Epoch 2/50
54000/54000 [==============================] - 4s - loss: 0.5026 - val_loss: 0.3277
Epoch 3/50
54000/54000 [==============================] - 4s - loss: 0.3921 - val_loss: 0.2692
Epoch 4/50
54000/54000 [==============================] - 4s - loss: 0.3475 - val_loss: 0.2427
Epoch 5/50
54000/54000 [==============================] - 4s - loss: 0.3327 - val_loss: 0.2302
Epoch 6/50
54000/54000 [==============================] - 4s - loss: 0.3066 - val_loss: 0.2081
Epoch 7/50
54000/54000 [==============================] - 4s - loss: 0.2981 - val_loss: 0.1982
Epoch 8/50
54000/54000 [==============================] - 4s - loss: 0.2885 - val_loss: 0.1967
Epoch 9/50
54000/54000 [==============================] - 4s - loss: 0.2855 - val_loss: 0.1803
Epoch 10/50
54000/54000 [==============================] - 4s - loss: 0.2752 - val_loss: 0.1835


In [70]:
y_train_pred = model.predict_classes(X_train, verbose = 0)

train_acc = np.sum(y_train == y_train_pred, axis=0)*1.0/X_train.shape[0]

print ("%.2f" % (train_acc*100))

y_test_pred = model.predict_classes(X_test, verbose = 0)

test_acc = np.sum(y_test == y_test_pred, axis=0)*1.0/X_test.shape[0]
print ("%.2f" % (test_acc*100))


96.16
95.85
